In [6]:
# %% [Setup]
import os
import shutil
import subprocess
import cv2
import numpy as np
import torch
import lpips
from skimage.metrics import structural_similarity as ssim

# Define paths
temp_folder = 'tmp_frames'
result_folder = 'results'

# Clean existing folders
for folder in [temp_folder, result_folder]:
    if os.path.isdir(folder):
        shutil.rmtree(folder)
    os.mkdir(folder)

print("Directories set up successfully!")


Directories set up successfully!


In [7]:
# %% [Select Video]
input_path = r'D:\University\IIT\Level 7\Final Year Project\MVP\EmotiLive\ESRGAN Model\inputs\ds3_1.mp4'  

if not os.path.exists(input_path):
    raise ValueError(f"Video file not found: {input_path}")

file_name = os.path.basename(input_path)
print(f"Selected Video: {file_name}")


Selected Video: ds3_1.mp4


In [8]:
print("Extracting frames from video...")

cmd = [
    'ffmpeg',
    '-i', input_path,
    '-vf', 'fps=15',  
    '-q:v', '1',      
    f'{temp_folder}/frame_%08d.png'
]

process = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
frame_count = len(os.listdir(temp_folder))

if process.returncode != 0 or frame_count == 0:
    raise RuntimeError("Error extracting frames")

print(f"Extracted {frame_count} frames successfully!")

Extracting frames from video...
Extracted 196 frames successfully!


In [10]:
print("Enhancing frames using Real-ESRGAN...")

cmd = [
    'python', 'inference_realesrgan.py',
    '-n', 'RealESRGAN_x4plus',
    '-i', temp_folder,
    '--outscale', '4',
    '--face_enhance'  
]

process = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

if process.returncode != 0:
    raise RuntimeError("Error enhancing frames")

print("Frame enhancement complete!")


Enhancing frames using Real-ESRGAN...
Frame enhancement complete!


In [11]:
import subprocess
import os

print("🎥 Recreating video from enhanced frames...")

output_video = os.path.join(result_folder, f"enhanced_{file_name}")
fps = 15  
cmd = [
    'ffmpeg',
    '-framerate', str(fps),
    '-i', os.path.join(temp_folder, 'frame_%08d.png'),  
    '-c:v', 'libx264',
    '-preset', 'slow',
    '-crf', '23',
    '-r', str(fps),
    '-pix_fmt', 'yuv420p',
    output_video
]


process = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Print the FFmpeg error message for debugging
print("FFmpeg Output:", process.stdout)
print("FFmpeg Error:", process.stderr)

if process.returncode != 0:
    raise RuntimeError(f"Error recreating video. FFmpeg Error:\n{process.stderr}")

print(f"Enhanced video saved as: {output_video}")


🎥 Recreating video from enhanced frames...
FFmpeg Output: 
FFmpeg Error: ffmpeg version N-118892-ge5d62e20c8-20250321 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (crosstool-NG 1.27.0.18_7458341)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --enable-shared --disable-static --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-libxml2 --enable-lzma --enable-fontconfig --enable-libharfbuzz --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --enable-libdvdread --enable-libdvdnav --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r -

In [12]:
# %% [Define Quality Metrics]
print("Initializing quality metrics")

lpips_loss = lpips.LPIPS(net='alex')  

def compute_psnr(image1, image2):
    mse = np.mean((image1 - image2) ** 2)
    return 100 if mse == 0 else 20 * np.log10(255.0 / np.sqrt(mse))

def compute_ssim(image1, image2):
    return ssim(image1, image2, data_range=255, channel_axis=-1)

def compute_mse(image1, image2):
    return np.mean((image1 - image2) ** 2)

def compute_lpips(image1, image2):
    image1 = torch.from_numpy(image1).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    image2 = torch.from_numpy(image2).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    return lpips_loss(image1, image2).item()

print("Quality metrics initialized!")


Initializing quality metrics
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


c:\Users\thavi\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\thavi\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: c:\Users\thavi\AppData\Local\Programs\Python\Python310\lib\site-packages\lpips\weights\v0.1\alex.pth
Quality metrics initialized!


In [13]:
# %% [Compare Videos]
def compare_videos(original_video_path, enhanced_video_path):
    original_video = cv2.VideoCapture(original_video_path)
    enhanced_video = cv2.VideoCapture(enhanced_video_path)

    psnr_values, ssim_values, mse_values, lpips_values = [], [], [], []
    frame_count = 0

    while original_video.isOpened() and enhanced_video.isOpened():
        ret1, frame1 = original_video.read()
        ret2, frame2 = enhanced_video.read()

        if not ret1 or not ret2:
            break  # Stop when no more frames

        frame1 = cv2.resize(frame1, (640, 640))
        frame2 = cv2.resize(frame2, (640, 640))

        frame_count += 1

        # Compute quality metrics
        psnr_values.append(compute_psnr(frame1, frame2))
        ssim_values.append(compute_ssim(frame1, frame2))
        mse_values.append(compute_mse(frame1, frame2))
        lpips_values.append(compute_lpips(frame1, frame2))

    original_video.release()
    enhanced_video.release()

    # Print Average Results
    print("\nVideo Quality Comparison Results")
    print(f"Average PSNR:  {np.mean(psnr_values):.2f} dB ")
    print(f"Average SSIM:  {np.mean(ssim_values):.4f} ")
    print(f"Average MSE:   {np.mean(mse_values):.2f} ")
    print(f"Average LPIPS: {np.mean(lpips_values):.4f}")

# Run comparison
compare_videos(input_path, output_video)



Video Quality Comparison Results
Average PSNR:  34.70 dB 
Average SSIM:  0.8792 
Average MSE:   25.52 
Average LPIPS: 0.0622
